# haystack和solr

- 安装配置好haystack，并且安装好solr后，根据下面的步骤，创建一个搜索引擎
- 安装pysolr用于使用Solr后端：pip install pysolr

## 开启solr并创建一个core

### 开启solr
- 执行完下面语句后，可以在[localhost:8983](http://localhost:8983/)打开solr管理界面

In [1]:
# 在cmd中
# 进入solr中的bin目录下
cd solr/bin
solr.cmd start # 不要以cloud的方式进行创建

### 创建一个core
- 创建core后可以在如下位置显示
![](img/h-s_1.png)

In [ ]:
'''
如果solr以cloud开启，则-c会默认创建一个collection
如果solr正常开启，则-c会默认创建一个core
'''
solr.cmd create -c core_name # 按照默认c的指定进行创建
solr.cmd create_core core_name # 创建一个core
solr.cmd create_collection core_name # 创建一个collection

'''
创建后core的目录结构：
core_name\
    - conf\
        - lang\
        - managed-schema
        - params.json
        - protwords.txt
        - solrconfig.xml
        - stopwords.txt
        - synonyms.txt
    - data\
    - core.properties
'''

## 配置

### 创建索引
- 在app\search_indexes.py文件中建立

In [ ]:
'''
searchIndex对象是Haystack确定应该在搜索引擎中放置哪些数据
并处理数据流的方式
'''

from haystack import indexes
from .models import Post

'''
通过这个索引，告诉Haystack这个模型中的哪些数据必须被搜索引擎编入索引
text：必须，主要搜索字段，use_template=True,将这个字段渲染成一个数据模板来构建document
published,title,body都是Post数据类中的属性，用于后面的索引使用
'''
class PostIndex(indexes.SearchIndex,indexes.Indexable):
    text = indexes.CharField(document=True,use_template=True)
    published = indexes.DateTimeField(model_attr='publish')
    title = indexes.CharField(model_attr='title')
    body = indexes.CharField(model_attr='body')

    def get_model(self):
        return Post

    def index_queryset(self, using=None):
        '''
        用于更新整个模型的索引
        :param using:
        :return:
        '''
        return self.get_model().published.all()

### 为text编写document模板
- 在templates/search/indexes/app_name/创建appname_text.txt文件

In [ ]:
# 文件内容，里面设置的内容为text的值
{{ object.title }}
{{ object.tag.all|join:',' }}
{{ object.body }}

### 将设置的索引配置到solr中

In [ ]:
# 创建出索引对应的xml语句
python manage.py build_solr_schema # 会在cmd中输出一段xml代码

'''
在代码中寻找所有和search_indexes设置相关的内容，包括下面的所有：
'''
'''
必有的内容，如果没有会自动创建，但创建后的django_ct和django_id默认为数组类型，会报错
'''
<field name="django_ct" type="string" indexed="true" stored="true" multiValued="false"/>
<field name="django_id" type="string" indexed="true" stored="true" multiValued="false"/>

'''
在PostIndex中设置的内容
'''
<field name="body" type="text_general" multiValued="false" indexed="true" stored="true"/>
<field name="published" type="pdate" multiValued="false" indexed="true" stored="true"/>
<field name="text" type="text_general" multiValued="false" indexed="true" stored="true"/>
<field name="title" type="string" multiValued="false" indexed="true" stored="true"/>



- 将上面的内容复制到managed-schema文件的filed区域
![](img/h-s_2.png)

In [ ]:
# 是否生成索引
python manage.py rebuild_index

### 检查上面的配置是否成功
1. 在solr-core-overview中的Num和Doc都有值
2. 在solr-core-query中点击查询有数据

## 视图展示

### 搜索表单

In [ ]:
# form.py文件
# 搜索视图
class SearchForm(forms.Form):
    query = forms.CharField()

### 搜索后台

In [ ]:
# view.py文件
# 搜索页面
def post_search(request):
    form = SearchForm()
    cd = None
    results = None
    total_results = None

    if 'query' in request.GET:
        form = SearchForm(request.GET)
        if form.is_valid():
            # 传入的请求内容{'query':xxx}
            cd = form.cleaned_data 
            # 搜索题目中含有query值的或文章主体含有query值的所有帖子
            results = SearchQuerySet().models(Post).filter(Q(title__contains=cd['query'])|Q(body__contains=cd['query'])).load_all()
            # 数量
            total_results = results.count()

    return render(request,'blog/post/search.html',{
        'form':form,
        'cd':cd,
        'results':results,
        'total_results':total_results
    })

In [ ]:
'''
filter：在django中默认是and语句来进行筛选
filter以or的方式进行筛选：
    引入from django.db.models import Q
    filter(Q(a='1') | Q(b)='2') ==== select * from xxx where a = 1 or b = 2
filter以not的方式进行筛选：
    引入from django.db.models import Q
    filter(~Q(c='123')) ==== select * from xxx where c != 123

filter可用的查询命令：注意双横线
    __exact 精确等于 like ‘aaa’
    __iexact 精确等于 忽略大小写 ilike ‘aaa’
    __contains 包含 like ‘%aaa%’
    __icontains 包含 忽略大小写 ilike ‘%aaa%’，但是对于sqlite来说，contains的作用效果等同于icontains。
    __gt 大于
    __gte 大于等于
    __lt 小于
    __lte 小于等于
    __in 存在于一个list范围内
    __startswith 以…开头
    __istartswith 以…开头 忽略大小写
    __endswith 以…结尾
    __iendswith 以…结尾，忽略大小写
    __range 在…范围内
    __year 日期字段的年份
    __month 日期字段的月份
    __day 日期字段的日
    __isnull=True/False
    __isnull=True
'''

### 前台界面

In [ ]:
{% load staticfiles %}
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Search</title>
    <link href="{% static "css/blog.css" %}" rel="stylesheet">
</head>
<body>
    <div id="content">
         {% if "query" in request.GET %}
			<h1>Posts containing "{{ cd.query }}"</h1>
			<h3>Found {{ total_results }} result{{ total_results|pluralize }}</h3>
			{% for result in results %}
				{% with post=result.object %}
					<h4><a href="{{ post.get_absolute_url }}">{{ post.title }}</a></h4>           
					{{ post.body}}
				{% endwith %}
				{% empty %}
					<p>There are no results for your query.</p>
			{% endfor %}    
			<p><a href="{% url "blog:post_search" %}">Search again</a></p>
		{% else %}
			<h1>Search for posts</h1>
			<form action="." method="get">
				{{ form.as_p }}
				<input type="submit" value="Search">
			</form>
		{% endif %}
    </div>
</body>
</html>

### 增加url

In [ ]:
# url.py文件中
path('search/',views.post_search,name='post_search')